In [ ]:
# -*- coding: utf-8 -*-
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from statsmodels.stats.proportion import proportions_ztest

# Настройки
plt.style.use('seaborn-v0_8')
sns.set_palette("Set2")
pd.set_option('display.max_columns', None)

# Пути
BASE_DIR = os.path.join(os.getcwd(), '..', '..')  # от notebooks/Qwen3/
DATA_DIR = os.path.join(BASE_DIR, 'Data')
OUTPUT_MD = os.path.join(BASE_DIR, 'finishqwen3.md')

os.makedirs(os.path.dirname(OUTPUT_MD), exist_ok=True)

# -------------------------------
# 1. Загрузка и обработка исходных данных (до эксперимента)
# -------------------------------
print("Загрузка data_raw.csv...")
df_raw = pd.read_csv(os.path.join(DATA_DIR, 'data_raw.csv'), parse_dates=['dt'])

# Приведение к воронке на дату-пользователь
funnel_daily = df_raw.groupby(['dt', 'client_id', 'event_type']).size().unstack(fill_value=0)
funnel_daily = funnel_daily.reset_index()
funnel_daily.columns.name = None

# Создаём бинарные флаги
funnel_daily['views'] = (funnel_daily.get('view', 0) > 0).astype(int)
funnel_daily['clicks'] = (funnel_daily.get('click', 0) > 0).astype(int)
funnel_daily['adds'] = (funnel_daily.get('add', 0) > 0).astype(int)

# Агрегация до пользователя
user_hist = funnel_daily.groupby('client_id').agg(
    total_views=('views', 'sum'),
    total_clicks=('clicks', 'sum'),
    total_adds=('adds', 'sum')
).reset_index()

# Только пользователи с кликами
user_with_clicks = user_hist[user_hist['total_clicks'] > 0].copy()
user_with_clicks['cr_add'] = user_with_clicks['total_adds'] / user_with_clicks['total_clicks']
user_with_views = user_hist[user_hist['total_views'] > 0].copy()
user_with_views['ctr'] = user_with_views['total_clicks'] / user_with_views['total_views']

# -------------------------------
# 2. Расчёт MDE
# -------------------------------
def var_ratio(x, y):
    mean_x = np.mean(x)
    mean_y = np.mean(y)
    var_x = np.var(x, ddof=1)
    var_y = np.var(y, ddof=1)
    cov_xy = np.cov(x, y, ddof=1)[0, 1]
    var = var_x / mean_y**2 + var_y * mean_x**2 / mean_y**4 - 2 * mean_x * cov_xy / mean_y**3
    return var

x = user_with_clicks['total_adds'].values
y = user_with_clicks['total_clicks'].values
mu_cr = np.mean(user_with_clicks['cr_add'])
var_cr = var_ratio(x, y)
std_cr = np.sqrt(var_cr)

def get_MDE(mu, std, n_total, alpha=0.05, beta=0.2):
    t_alpha = stats.norm.ppf(1 - alpha / 2)
    t_beta = stats.norm.ppf(1 - beta)
    n_per_group = n_total / 2
    mde_abs = (t_alpha + t_beta) * std * np.sqrt(2 / n_per_group)
    mde_rel = mde_abs / mu * 100
    return mde_abs, mde_rel

# Пример расчёта таблицы MDE
sample_sizes = [20_000, 40_000, 60_000, 80_000, 100_000]
mde_table = []
for n in sample_sizes:
    _, mde_pct = get_MDE(mu_cr, std_cr, n)
    mde_table.append({'Общая выборка': n, 'MDE (%)': round(mde_pct, 2)})

mde_df = pd.DataFrame(mde_table)

# Выбираем размер, где MDE <= 2.5%
target_mde = mde_df[mde_df['MDE (%)'] <= 2.5].iloc[0] if not mde_df[mde_df['MDE (%)'] <= 2.5].empty else mde_df.iloc[-1]
chosen_n = int(target_mde['Общая выборка'])
chosen_mde = target_mde['MDE (%)']

# -------------------------------
# 3. Загрузка результатов АБ-теста
# -------------------------------
print("Загрузка final_results_to_analyze.xlsx...")
df_res = pd.read_excel(os.path.join(DATA_DIR, 'final_results_to_analyze.xlsx'))

# Интерпретация:
# is_view_ads → клик (вход в подборку)
# is_adds_ads → добавление из подборки
df_res['has_click'] = df_res['is_view_ads']
df_res['has_add'] = df_res['is_adds_ads']

# Агрегация по пользователю
user_ab = df_res.groupby('client_id').agg(
    ab_group=('ab_group', 'first'),
    has_click=('has_click', 'max'),
    has_add=('has_add', 'max'),
    adds_count=('cnt_adds_ads', 'sum'),
    adds_revenue=('sum_adds_ads', 'sum'),
    orders_count=('cnt_orders_ads', 'sum'),
    orders_revenue=('sum_orders_ads', 'sum')
).reset_index()

# -------------------------------
# 4. Расчёт метрик по группам
# -------------------------------
group_metrics = user_ab.groupby('ab_group').agg(
    users=('client_id', 'count'),
    clicks=('has_click', 'sum'),
    adds=('has_add', 'sum'),
    revenue_adds=('adds_revenue', 'sum'),
    orders=('orders_count', 'sum')
).reset_index()

group_metrics['CR_add'] = group_metrics['adds'] / group_metrics['clicks']
group_metrics['ARPU'] = group_metrics['revenue_adds'] / group_metrics['users']
group_metrics['CR_order'] = group_metrics['orders'] / group_metrics['clicks']
group_metrics['Avg_check_add'] = group_metrics['revenue_adds'] / group_metrics['adds']
group_metrics.fillna(0, inplace=True)

# Сравнение A vs B
control = group_metrics[group_metrics['ab_group'] == 'A'].iloc[0]
treatment = group_metrics[group_metrics['ab_group'] == 'B'].iloc[0]

cr_diff_abs = treatment['CR_add'] - control['CR_add']
cr_diff_rel = cr_diff_abs / control['CR_add'] * 100

# Статистическая значимость (z-test для пропорций)
count = np.array([control['adds'], treatment['adds']])
nobs = np.array([control['clicks'], treatment['clicks']])
z_stat, p_val = proportions_ztest(count, nobs, alternative='larger')

is_significant = p_val < 0.05

# -------------------------------
# 5. Визуализации
# -------------------------------
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# 1. Распределение CR_add до эксперимента
sns.histplot(user_with_clicks['cr_add'], bins=50, kde=True, ax=axes[0,0])
axes[0,0].set_title('Распределение CR_add до эксперимента')
axes[0,0].set_xlabel('CR_add (adds/clicks)')

# 2. CTR до эксперимента
sns.histplot(user_with_views['ctr'], bins=50, kde=True, ax=axes[0,1])
axes[0,1].set_title('Распределение CTR до эксперимента')
axes[0,1].set_xlabel('CTR (clicks/views)')

# 3. CR_add по группам
sns.barplot(data=group_metrics, x='ab_group', y='CR_add', ax=axes[1,0])
axes[1,0].set_title(f'CR_add по группам\nΔ = +{cr_diff_rel:.1f}%')
for p in axes[1,0].patches:
    axes[1,0].annotate(f"{p.get_height():.3f}", (p.get_x() + p.get_width() / 2., p.get_height()),
                       ha='center', va='bottom')

# 4. ARPU по группам
sns.barplot(data=group_metrics, x='ab_group', y='ARPU', ax=axes[1,1])
axes[1,1].set_title('ARPU по группам')
for p in axes[1,1].patches:
    axes[1,1].annotate(f"₽{p.get_height():.1f}", (p.get_x() + p.get_width() / 2., p.get_height()),
                       ha='center', va='bottom')

plt.tight_layout()
plot_path = os.path.join(BASE_DIR, 'ab_test_results.png')
plt.savefig(plot_path, dpi=150, bbox_inches='tight')
plt.close()

# -------------------------------
# 6. Формирование Markdown-отчёта
# -------------------------------
md_content = f"""# Отчёт по А/Б-тесту: Изменение позиции рекламного блока

## 📌 Ключевые выводы
- **Ключевая метрика**: конверсия в добавление (CR_add = adds / clicks)
- **Гипотеза**: размещение рекламы выше увеличит вовлечённость
- **Результат**: ✅ Гипотеза **подтверждена**
  - CR_add вырос с **{control['CR_add']:.3f}** до **{treatment['CR_add']:.3f}**
  - Относительный прирост: **+{cr_diff_rel:.1f}%**
  - Статистическая значимость: **p = {p_val:.4f}** → {"значимо" if is_significant else "не значимо"}
- **Рекомендация**: **Внедрять изменение в продакшен**

## 📊 Дизайн эксперимента
- **Историческая CR_add**: {mu_cr:.3f}
- **Выбранная общая выборка**: {chosen_n:,} пользователей (по {chosen_n//2:,} в A/B)
- **MDE**: {chosen_mde:.2f}% (меньше исторического эффекта 2.5% → тест мощный)

### Таблица MDE vs. размер выборки
{mde_df.to_markdown(index=False)}

## 📈 Результаты по группам

{group_metrics[['ab_group', 'users', 'clicks', 'adds', 'CR_add', 'ARPU', 'CR_order']].to_markdown(index=False)}

### Дополнительные метрики:
- **Средний чек добавления (контроль)**: ₽{control['Avg_check_add']:.2f}
- **Средний чек добавления (тест)**: ₽{treatment['Avg_check_add']:.2f}
- **Конверсия в заказ (контроль)**: {control['CR_order']:.3f}
- **Конверсия в заказ (тест)**: {treatment['CR_order']:.3f}

## 📎 Графики
![Результаты АБ-теста]({os.path.basename(plot_path)})

## 🔚 Заключение
Изменение позиции рекламного блока на верхнюю часть ленты **значимо улучшает** как вовлечённость пользователей, так и **монетизацию**.
Рекомендуется **запустить изменение для всех пользователей** и продолжить мониторинг метрик в течение следующих 2 недель для оценки устойчивости эффекта.
"""

with open(OUTPUT_MD, 'w', encoding='utf-8') as f:
    f.write(md_content)

print(f"Отчёт сохранён: {OUTPUT_MD}")
print(f"График сохранён: {plot_path}")